# ETTm1 Time Series Forecasting

This notebook runs and compares four forecasting models on the ETTm1 dataset:
1. **Seasonal Naive** - Baseline using seasonal patterns
2. **DLinear** - Decomposition + Linear layers
3. **LSTM** - Recurrent neural network baseline
4. **Mamba** - State space model for sequence modeling

## Setup
1. Runtime -> Change runtime type -> **GPU** (T4 recommended)
2. Run all cells in order

---
## 1. Environment Setup

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: No GPU detected! Enable GPU: Runtime -> Change runtime type -> GPU")

In [ ]:
# Clone repository (skip if already cloned)
import os
if not os.path.exists('Mamba-ETTm1-Forecasting'):
    !git clone https://github.com/steinpleiter/Mamba-ETTm1-Forecasting.git
    %cd Mamba-ETTm1-Forecasting
else:
    %cd Mamba-ETTm1-Forecasting
    !git pull origin main

In [ ]:
# Install dependencies
!pip install -q pandas numpy scikit-learn matplotlib seaborn pyyaml tqdm

# Install Mamba dependencies (requires specific PyTorch version)
!pip install -q torch==2.2.1 --index-url https://download.pytorch.org/whl/cu121
!pip install -q packaging ninja
!pip install -q causal-conv1d==1.4.0
!pip install -q mamba-ssm==2.2.2

# Ensure numpy compatibility
!pip install -q "numpy<2.0"

print("\nDependencies installed!")

In [ ]:
# Verify installations
import numpy as np
import torch
from mamba_ssm import Mamba

print(f"NumPy: {np.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

# Quick Mamba test
m = Mamba(d_model=64).cuda()
x = torch.randn(1, 10, 64).cuda()
_ = m(x)
print("Mamba: OK")

---
## 2. Download and Preprocess Data

In [ ]:
# Download ETTm1 dataset
import urllib.request
os.makedirs('data/raw', exist_ok=True)

url = 'https://raw.githubusercontent.com/zhouhaoyi/ETDataset/main/ETT-small/ETTm1.csv'
urllib.request.urlretrieve(url, 'data/raw/ETTm1.csv')
print("Dataset downloaded: data/raw/ETTm1.csv")

In [ ]:
# Preprocess data
!python scripts/preprocess_data.py --config configs/base_config.yaml

---
## 3. Model Training & Evaluation

### 3.1 Seasonal Naive Baseline

In [ ]:
# Evaluate Seasonal Naive baseline
!python scripts/evaluate_baseline.py \
    --model seasonal_naive \
    --device cuda \
    --save_results

### 3.2 DLinear Model

In [ ]:
# Train DLinear (channel-independent mode)
!python scripts/train_dlinear.py \
    --device cuda \
    --individual \
    --epochs 50 \
    --batch_size 64 \
    --config configs/base_config.yaml

### 3.3 LSTM Model

In [ ]:
# Train LSTM
!python scripts/train_lstm.py \
    --device cuda \
    --epochs 50 \
    --batch_size 64 \
    --hidden_size 128 \
    --num_layers 2 \
    --config configs/base_config.yaml

### 3.4 Mamba Model

In [ ]:
# Train Mamba
!python scripts/train_mamba.py \
    --config configs/base_config.yaml \
    --device cuda \
    --epochs 50 \
    --batch_size 32 \
    --d_model 128 \
    --n_layers 4 \
    --patch_len 16

---
## 4. Model Comparison

In [ ]:
# Compare all trained models
!python scripts/compare_models.py

In [ ]:
# Display comparison plots
from IPython.display import Image, display
from pathlib import Path

fig_path = Path('results/figures')

if (fig_path / 'model_comparison.png').exists():
    display(Image(filename=str(fig_path / 'model_comparison.png')))

if (fig_path / 'final_performance.png').exists():
    display(Image(filename=str(fig_path / 'final_performance.png')))

---
## 5. Results Summary

In [ ]:
# Load and display results
import pickle
import pandas as pd

def load_results(filename):
    path = Path('results') / filename
    if path.exists():
        with open(path, 'rb') as f:
            return pickle.load(f)
    return None

# Collect results
results = []

models = {
    'Seasonal Naive': 'seasonal_naive_baseline_results.pkl',
    'DLinear': 'dlinear_ETTm1_training_results.pkl',
    'LSTM': 'lstm_ETTm1_training_results.pkl',
    'Mamba': 'mamba_ETTm1_training_results.pkl'
}

for name, filename in models.items():
    data = load_results(filename)
    if data:
        metrics = data.get('test_metrics', data.get('val_metrics', {}))
        results.append({
            'Model': name,
            'MAE': metrics.get('mae', float('nan')),
            'RMSE': metrics.get('rmse', float('nan')),
            'MASE': metrics.get('mase', float('nan')),
            'Parameters': data.get('n_parameters', 0)
        })

if results:
    df = pd.DataFrame(results)
    df = df.sort_values('MAE')
    print("\nModel Performance (sorted by MAE):")
    print("=" * 70)
    print(df.to_string(index=False))
    
    # Calculate improvement over baseline
    baseline_mae = df[df['Model'] == 'Seasonal Naive']['MAE'].values[0]
    df['Improvement'] = ((baseline_mae - df['MAE']) / baseline_mae * 100).round(2)
    df['Improvement'] = df['Improvement'].apply(lambda x: f"+{x:.1f}%" if x > 0 else f"{x:.1f}%")
    
    print("\n\nImprovement over Seasonal Naive:")
    print("=" * 70)
    print(df[['Model', 'MAE', 'Improvement']].to_string(index=False))
else:
    print("No results found. Run the training cells first.")

---
## 6. Download Results

In [ ]:
# Package results for download
!zip -r results.zip results/

from google.colab import files
files.download('results.zip')
print("\nDownload started! Extract results.zip on your local machine.")